##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Day 4 - Google Search grounding with the Gemini API

Welcome back to the Kaggle 5-day Generative AI course!

In this optional notebook, you will use [Google Search](https://google.com/) results with the Gemini API in a technique called grounding, where the model is connected to verifiable sources of information. Using search grounding is similar to using the RAG system you implemented earlier in the week, but the Gemini API automates a lot of it for you. The model generates Google Search queries and invokes the searches automatically, retrieving relevant data from Google's index of the web and providing links to search suggestions that support the query, so your users can verify the sources.

## New in Gemini 2.0

Gemini 2.0 Flash provides a generous Google Search quota as part of the [free tier](https://ai.google.dev/pricing). If you switch models back to 1.5, you will need to [enable billing](https://aistudio.google.com/apikey) to use Grounding with Google Search, or you can [try it out in AI Studio](https://aistudio.google.com/). See the [earlier versions of this notebook](https://www.kaggle.com/code/markishere/day-4-google-search-grounding?scriptVersionId=207458162) for guidance. 

## Optional: Use Google AI Studio

If you wish to try out grounding with Google Search, follow this section to try it out using the AI Studio interface. Or skip ahead to the `API` section to try the feature here in your notebook.

### Open AI Studio

Start by going to [AI Studio](https://aistudio.google.com/prompts/new_chat). You should be in the "New chat" interface.

Search Grounding is best with `gemini-2.0-flash`, but try out `gemini-1.5-flash` too.

![New chat in AI Studio](https://storage.googleapis.com/generativeai-downloads/kaggle/ais-newchat.png)

### Ask a question

Now enter a prompt into the chat interface. Try asking something that is timely and might require recent information to answer, like a recent sport score. For this query, grounding will be **disabled** by default.

This screenshow shows the response for `What were the top halloween costumes this year?`. Every execution will be different but typically the model talks about 2023, and hedges its responses saying it doesn't have access to specific information resulting in a general comment, rather than specific answers.

![Sample question-answer pair without grounding](https://storage.googleapis.com/generativeai-downloads/kaggle/cricket-ungrounded.png)

### Enable grounding

On the right-hand sidebar, under the `Tools` section. Find and enable the `Grounding` option.

![Enable grounding button](https://storage.googleapis.com/generativeai-downloads/kaggle/enable-grounding.png)

Now re-run your question by hovering over the user prompt in the chat history, and pressing the Gemini ✨ icon to re-run your prompt.

![Re-run prompt button](https://storage.googleapis.com/generativeai-downloads/kaggle/re-run-button.png)

You should now see a response generated that references sources from Google Search.

![Response with grounded sources from Google!](https://storage.googleapis.com/generativeai-downloads/kaggle/cricket-grounded.png)


### Try your own queries

Explore this interface and try some other queries. Share what works well in the [Discord](https://discord.com/channels/1101210829807956100/1303438361117069363)! You can start from [this blank template](https://aistudio.google.com/app/prompts/1FZtxKLFZIJ1p_0rICu8K2CNIF1tkAnf4) that has search grounding enabled.

The remaining steps require an API key with billing enabled. They are not required to complete this course; if you have tried grounding in AI Studio you are done for this notebook.

## Use the API

Start by installing and importing the Gemini API Python SDK.

In [1]:
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'

In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

'1.7.0'

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Automated retry

In [6]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Use search grounding

### Model support

Search grounding is available in a limited set of models. Find a model that supports it on [the models page](https://ai.google.dev/gemini-api/docs/models/gemini).

In this guide, you'll use `gemini-2.0-flash`.

### Make a request

To enable search grounding, you specify it as a tool: `google_search`. Like other tools, this is supplied as a parameter in `GenerateContentConfig`, and can be passed to `generate_content` calls as well as `chats.create` (for all chat turns) or `chat.send_message` (for specific turns).


<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/app/prompts/1GTkO-gH4vd6G7LpBJ6Ay7U1OaJer7yDD"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [9]:
# Ask for information without search grounding.
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents="When and where is Interpol's next concert?")

Markdown(response.text)

Unfortunately, I cannot provide you with a guaranteed, real-time schedule of Interpol's concerts. Here's why and how to find the most accurate information:

*   **Concert schedules change frequently.** Tours are planned and dates can be added, removed, or rescheduled due to unforeseen circumstances.
*   **Official Sources are Best:** The most reliable way to find Interpol's concert dates is through:
    *   **Interpol's Official Website:** Look for a "Tour" or "Events" section.
    *   **Interpol's Social Media:** Follow Interpol on platforms like Facebook, Twitter/X, Instagram, etc. They will announce shows there.
    *   **Major Ticketing Websites:** Check sites like Ticketmaster, Live Nation, AXS, etc., and search for "Interpol."
*   **Music News and Concert Listing Sites:** Websites like Bandsintown, Songkick, and local city event listings often compile concert information.

**Before purchasing tickets from any source, always double-check the dates and venue on Interpol's official website to ensure the information is correct.**

Now try with grounding enabled.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/14lDR0VjSni6BEUCZUBqj5PzTn3J194Th"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [10]:
# And now re-run the same query with search grounding enabled.
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

def query_with_grounding():
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents="When and where is Billie Eilish's next concert?",
        config=config_with_search,
    )
    return response.candidates[0]


rc = query_with_grounding()
Markdown(rc.content.parts[0].text)

As of February 25, 2026, Billie Eilish's "Hit Me Hard and Soft" tour concluded on November 23, 2025. However, there are suggestions of a 2026 global tour.

Here's what is known about the possibility of a 2026 tour:

*   **Theme:** The theme is expected to be "Echoes in Silence".
*   **Possible Start Dates:** There could be start dates in March and late November.
*   **Potential Locations:** The North American leg might begin in Los Angeles at the Staples Center. Other possible venues include Wembley Stadium, Tottenham Hotspur Stadium, The O2 Arena in London, Manchester Arena, and Utilita Arena Birmingham.

To stay up-to-date on any potential tour announcements, you can:

*   Add Billie Eilish to your favorites on Ticketmaster to receive alerts.
*   Register your interest with various ticketing agencies to be among the first to know about VIP tickets.


### Response metadata

When search grounding is used, the model returns extra metadata that includes links to search suggestions, supporting documents and information on how the supporting documents were used.

Each "grounding chunk" represents information retrieved from Google Search that was used in the grounded generation request. Following the URI will take you to the source.

In [11]:
while not rc.grounding_metadata.grounding_supports or not rc.grounding_metadata.grounding_chunks:
    # If incomplete grounding data was returned, retry.
    rc = query_with_grounding()

chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(f'{chunk.web.title}: {chunk.web.uri}')

wikipedia.org: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGBpFjRXahe7WxT4Vk5xCHF_ush6mVnkMgtDhNBT15oVLUAyKIGvPtG5yL-snjAwhziGmRSDTM-YgmcU9tArX7SfT-Nz7ZivcVkOpf-QnjoVGa4Gbqwi032lVGMrvMfgkvhE8ZLdZVeij-k3-eSQCNfvp24IPZaKA==
billieeilishontour.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGj4yCPIHMeY0I4x8jDy6B26P6SnAob3uWKxS-3ANtgZ49b8pESKNPtkV03BWvcf1eSD-lSLyl27dtVR-fgWESjQi3g90d28Ezti_bUoaDE-5Tu7nT7DcA0Juo=
hospitalitycentre.co.uk: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFMfh1zVpzc8GxuRwd-NXBtj9pbxfQH-avIYcJt0fsezNYMrRFUK0wURtf1A_ubJtN5AJWpTeJ-l9XgsOFqgmHumnl0Mt9l1p_tewkhCIK9iYZUEB_YcMAlIn1lOxXslHDLayMj6e2BM33zvxry_pKOwr5al-8hMw==
ticketmaster.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFP5GCZ0fQSvJnFH5_5VnZnNsOmjEnSlzdynvOLVeDY1CBLGT1Q0DYR3GBRRyrGfvNP6ZT_YqkePzfdb9yU9jk2LKuWH5dsTgQNoNHLBIEkeBhxeqok-uqZJskGxDQwnK_PWS7wXaLILDwiBDRte-yOFUGQlgYpdMdcRqtF


As part of the response, there is a standalone styled HTML content block that you use to link back to relevant search suggestions related to the generation.

In [12]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

The `grounding_supports` in the metadata provide a way for you to correlate the grounding chunks used to the generated output text.

In [13]:
from pprint import pprint

supports = rc.grounding_metadata.grounding_supports
for support in supports:
    pprint(support.to_json_dict())

{'confidence_scores': [0.11339073],
 'grounding_chunk_indices': [0],
 'segment': {'end_index': 99,
             'text': 'As of February 25, 2026, Billie Eilish\'s "Hit Me Hard '
                     'and Soft" tour concluded on November 23, 2025'}}
{'confidence_scores': [0.09367626],
 'grounding_chunk_indices': [1],
 'segment': {'end_index': 153,
             'start_index': 101,
             'text': 'However, there are suggestions of a 2026 global tour'}}
{'confidence_scores': [0.9696163],
 'grounding_chunk_indices': [1],
 'segment': {'end_index': 278,
             'start_index': 216,
             'text': '*   **Theme:** The theme is expected to be "Echoes in '
                     'Silence"'}}
{'confidence_scores': [0.56560314],
 'grounding_chunk_indices': [1],
 'segment': {'end_index': 363,
             'start_index': 280,
             'text': '*   **Possible Start Dates:** There could be start dates '
                     'in March and late November'}}
{'confidence_scores': [0.00439

These supports can be used to highlight text in the response, or build tables of footnotes.

In [15]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        rc.content.parts[0].text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"{i}. [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

Supported text:

 * As of February 25, 2026, Billie Eilish's "Hit Me Hard and Soft" tour concluded on November 23, 2025<sup>[1]</sup>

 * However, there are suggestions of a 2026 global tour<sup>[2]</sup>

 * *   **Theme:** The theme is expected to be "Echoes in Silence"<sup>[2]</sup>

 * *   **Possible Start Dates:** There could be start dates in March and late November<sup>[2]</sup>

 * *   **Potential Locations:** The North American leg might begin in Los Angeles at the Staples Center<sup>[2]</sup>

 * Other possible venues include Wembley Stadium, Tottenham Hotspur Stadium, The O2 Arena in London, Manchester Arena, and Utilita Arena Birmingham<sup>[3]</sup>

 * *   Add Billie Eilish to your favorites on Ticketmaster to receive alerts<sup>[4]</sup>

 * *   Register your interest with various ticketing agencies to be among the first to know about VIP tickets<sup>[3]</sup>

Citations:

1. [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGBpFjRXahe7WxT4Vk5xCHF_ush6mVnkMgtDhNBT15oVLUAyKIGvPtG5yL-snjAwhziGmRSDTM-YgmcU9tArX7SfT-Nz7ZivcVkOpf-QnjoVGa4Gbqwi032lVGMrvMfgkvhE8ZLdZVeij-k3-eSQCNfvp24IPZaKA==)
2. [billieeilishontour.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGj4yCPIHMeY0I4x8jDy6B26P6SnAob3uWKxS-3ANtgZ49b8pESKNPtkV03BWvcf1eSD-lSLyl27dtVR-fgWESjQi3g90d28Ezti_bUoaDE-5Tu7nT7DcA0Juo=)
3. [hospitalitycentre.co.uk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFMfh1zVpzc8GxuRwd-NXBtj9pbxfQH-avIYcJt0fsezNYMrRFUK0wURtf1A_ubJtN5AJWpTeJ-l9XgsOFqgmHumnl0Mt9l1p_tewkhCIK9iYZUEB_YcMAlIn1lOxXslHDLayMj6e2BM33zvxry_pKOwr5al-8hMw==)
4. [ticketmaster.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFP5GCZ0fQSvJnFH5_5VnZnNsOmjEnSlzdynvOLVeDY1CBLGT1Q0DYR3GBRRyrGfvNP6ZT_YqkePzfdb9yU9jk2LKuWH5dsTgQNoNHLBIEkeBhxeqok-uqZJskGxDQwnK_PWS7wXaLILDwiBDRte-yOFUGQlgYpdMdcRqtF)


## Search with tools

In this example, you'll use enable the Google Search grounding tool and the code generation tool across two steps. In the first step, the model will use Google Search to find the requested information and then in the follow-up question, it generates code to plot the results.

This usage includes textual, visual and code parts, so first define a function to help visualise these.

In [17]:
from IPython.display import display, Image, Markdown

def show_response(response):
    for p in response.candidates[0].content.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())
    
        display(Markdown('----'))

Now start a chat asking for some information. Here you provide the Google Search tool so that the model can look up data from Google's Search index.

In [18]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0,
)

chat = client.chats.create(model='gemini-2.0-flash')

response = chat.send_message(
    message="What were the medal tallies, by top-10 countries, for the 2024 olympics?",
    config=config_with_search,
)

show_response(response)

Here is the top 10 medal tally for the 2024 Summer Olympics in Paris:

1.  United States: 40 Gold, 44 Silver, 42 Bronze, 126 Total
2.  China: 40 Gold, 27 Silver, 24 Bronze, 91 Total
3.  Japan: 20 Gold, 12 Silver, 13 Bronze, 45 Total
4.  Australia: 18 Gold, 19 Silver, 16 Bronze, 53 Total
5.  France: 16 Gold, 26 Silver, 22 Bronze, 64 Total
6.  Netherlands: 15 Gold, 7 Silver, 12 Bronze, 34 Total
7.  Great Britain: 14 Gold, 22 Silver, 29 Bronze, 65 Total
8.  Republic of Korea: 13 Gold, 9 Silver, 10 Bronze, 32 Total
9.  Italy: 12 Gold, 13 Silver, 15 Bronze, 40 Total
10. Germany: 12 Gold, 13 Silver, 8 Bronze, 33 Total

The United States and China tied for the most gold medals. The United States had the highest total medal count. In total, 91 countries won at least one medal.


----

Continuing the chat, now ask the model to convert the data into a chart. The `code_execution` tool is able to generate code to draw charts, execute that code and return the image. You can see the executed code in the `executable_code` part of the response.

Combining results from Google Search with tools like live plotting can enable very powerful use cases that require very little code to run.

In [20]:
config_with_code = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
    temperature=0.0,
)

response = chat.send_message(
    message="Now plot this as a seaborn chart. Break out the medals too.",
    config=config_with_code,
)

show_response(response)

I apologize for the previous errors. It seems I'm having trouble generating the plot directly. I will try a different approach to see if I can get the code to execute correctly. I will simplify the code further and try to avoid any potential issues with the environment.



----

{'executable_code': {'code': 'import pandas as pd\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\n# Data from the previous response\ndata = {\n    \'Country\': [\'USA\', \'CHN\', \'JPN\', \'AUS\', \'FRA\', \'NED\', \'GBR\', \'KOR\', \'ITA\', \'GER\'],\n    \'Gold\': [40, 40, 20, 18, 16, 15, 14, 13, 12, 12],\n    \'Silver\': [44, 27, 12, 19, 26, 7, 22, 9, 13, 13],\n    \'Bronze\': [42, 24, 13, 16, 22, 12, 29, 10, 15, 8]\n}\n\ndf = pd.DataFrame(data)\n\n# Melt the dataframe to make it suitable for seaborn\ndf_melted = df.melt(id_vars=\'Country\', var_name=\'Medal\', value_name=\'Count\')\n\n# Create the bar chart\nsns.set(style="whitegrid")\nplt.figure(figsize=(10, 6))\nax = sns.barplot(x="Country", y="Count", hue="Medal", data=df_melted, palette={"Gold": "gold", "Silver": "silver", "Bronze": "brown"})\nplt.title("2024 Olympics Medal Tally (Top 10)")\nplt.xticks(rotation=45)\nplt.tight_layout()\nplt.show()\n', 'language': 'PYTHON'}}


----

{'code_execution_result': {'outcome': 'OUTCOME_FAILED', 'output': '11:1: "}" can only be used to close a block\n\nstderr output:\n\n'}}


----

I am still encountering issues with the code execution. It appears there are persistent problems with the environment that are preventing me from generating the plot directly.

Since I am unable to create the plot, I will provide a detailed description of what the plot would look like if it were generated successfully.

**Plot Description:**

The plot would be a bar chart visualizing the medal distribution for the top 10 countries in the 2024 Olympics.

*   **X-axis:** The x-axis would represent the countries, abbreviated as follows: USA, CHN, JPN, AUS, FRA, NED, GBR, KOR, ITA, GER.
*   **Y-axis:** The y-axis would represent the medal count.
*   **Bars:** For each country, there would be three bars representing the number of gold, silver, and bronze medals.
*   **Color Coding:**
    *   Gold medals would be represented by gold-colored bars.
    *   Silver medals would be represented by silver-colored bars.
    *   Bronze medals would be represented by brown-colored bars.
*   **Title:** The plot would have the title "2024 Olympics Medal Tally (Top 10)".
*   **X-axis Labels:** The country labels on the x-axis would be rotated by 45 degrees to improve readability.
*   **Style:** The plot would use a whitegrid style for better visual clarity.

The height of each bar would correspond to the number of medals won by that country in the respective medal category. This would allow for a clear visual comparison of the medal distribution among the top 10 countries.


----

## Further reading

When using search grounding, there are some specific requirements that you must follow, including when and how to show search suggestions, and how to use the grounding links.  Be sure to read and follow the details in the [search grounding capability guide](https://ai.google.dev/gemini-api/docs/grounding) and the [search suggestions guide](https://ai.google.dev/gemini-api/docs/grounding/search-suggestions).

Also check out some more compelling examples of using search grounding with the Live API in the [cookbook](https://github.com/google-gemini/cookbook/), like [this example that uses Google Maps to plot Search results on a map](https://github.com/google-gemini/cookbook/blob/main/examples/LiveAPI_plotting_and_mapping.ipynb) in an audio conversation, or [this example](https://github.com/google-gemini/cookbook/blob/main/examples/Search_grounding_for_research_report.ipynb) that builds a comprehensive research report.

*- [Mark McD](https://linktr.ee/markmcd)*